In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D

NCHANNELS = 64
#all distances in millimeters, times in nanoseconds
ZCELL = 13 #height of a cell
z_pts = [ZCELL*0.5,ZCELL*1.5,ZCELL*2.5,ZCELL*3.5] #4 possible Z-coordinate values for a wire
XCELL = 42 #width of a cell
VDRIFT = .054 #drift velocity
Z_SEP = 784 #distance between the upper and lower chambers (measured from base of chamber 1 to base of chamber 3)
jitter = .55 #aproximate adjustment for difference between recorded and 'actual' x/y coordinates of hits

#various parameters for acceptance cuts, adjust as you see fit (these values seemed to be a good balance of qulaity vs quantity to me)
max_hits = 99 #maximum allowed number of hits in one event (Note:Higher multiplicity events can take extremely long to process, so you might want to set a low maximum to exclude them)
max_slope = float(np.sqrt(3)) #maximum allowed slope from the vertical
sigma = .4 #expected uncertainty in local reconstructions
chisq_best = 20. #max acceptable chi squared for local reconstructions
cut2d = 200. #max acceptable chi squared for 2d global reconstructions (Note:Not calculated with sigma,primarily adjust this for global reconstructions)
cut3d = 500. #max acceptable chi squared for 3d global reconstructions (Note:Not calculated with sigma,less impactful than changing cut2d)

#helper function to remove unneeded zero entries from arrays
def removezeros(arr):
    zeros = np.all(np.equal(arr, 0), axis=1)
    arr = arr[~zeros]
    return arr

#checks if the slope between the start and end point is greater than the maximum allowed value
def allowed_slope(xs,ys):
    if np.around(ys[-1]-ys[0]) == 0:
        slope = 100
    else:
        slope = abs(float((xs[-1]-xs[0])/(ys[-1]-ys[0])))
    if slope > max_slope:
        return False
    else:
        return True
    
#checks to see if two tracks have any points in common   
def common(x,entries):
    i = 0
    for entry in entries:
        x_set = set(x) 
        entry_set = set(entry) 
        if (x_set & entry_set): 
            #if there is a point in common between x and a given entry, the index of the entry which has the point in common is returned
            return i 
        i += 1
    return 100 #returned to indicate that no entries had any points in common with x 

#checks to see if two tracks are the XLEFT and XRIGHT versions of each other
def opposites(x1,x2):
    i = 0
    for entry in x2:
        opposite = True
        for j in np.arange(len(entry)):
            try:
                if np.round(x1[j]+entry[j]) % 42 != 0:
                    opposite =  False
            except:
                #an exception occurs if the entry is empty, in which case x1 cannot be its opposite
                opposite = False
        if opposite:
            #if there is an entry in x2 which is the opposite of x1, the index of that entry is returned
            return i
        i += 1
    return 100 #returned to indicate that no entries in x2 had any points in common with x1

#looks at all possible combinations of points and creates the best possible line of fit
def find_fit(df):
    #group hits by their y-coordinate so that only one hit from each height will be selected
    chambs = df.groupby('y')
    list1 = []
    #create a list of all possible combinations of points
    for i,chamb in chambs:
        list1.append(chamb.to_numpy())
    points = np.array(list(itertools.product(*list1)))

    #iterate over all possible combinations and return a list of fits that are potential muon tracks
    dof = 2 #degrees of freedom in the fit
    #initialize some arrays to hold information for successful fits
    x_fits = []
    x_best = []
    y_best = []
    chisquares = []
    cleaned = []
    angle = 1 #keeps track of whether an acceptable angle was found or not
    count = np.arange(len(points))
    for i in count:
        #use RLM (robust liner model) to find a fit for each combination, if an acceptable one exists
        xs = [x[0] for x in points[i]]
        ys = [x[1] for x in points[i]]
        xs_clean = []
        ys_clean = []
        clean = 0 #keeps track of whether the entry was 'cleaned' of outliers or not
        try:
            reg = sm.RLM(xs,sm.add_constant(ys)).fit(update_scale = False)
            chisq = np.sum(((reg.resid)/sigma)** 2)
            weights = reg.weights #points which RLM detected as outliers are given a weight > 1
            fit = list(reversed(reg.params))
            x_fit = list(np.poly1d(fit)(z_pts))

            # if the fit is not good, try again while ignoring points RLM has marked as outliers
            if chisq/dof > chisq_best:
                for i in np.arange(len(xs)):
                    if weights[i] == 1:
                        #make a new array consisting of only points which were not weighted as outliers
                        xs_clean.append(xs[i])
                        ys_clean.append(ys[i])
                clean = 1 #indicates the event was 'cleaned'
                if len(xs_clean) < 3: #skips combinations which have too few hits left after being 'cleaned'
                    continue
                try:
                    reg = sm.RLM(xs_clean,sm.add_constant(ys_clean)).fit(update_scale = False)
                    fit = list(reversed(reg.params))
                    chisq = np.sum(((reg.resid)/sigma)** 2)
                    x_fit = list(np.poly1d(fit)(z_pts))
                except ZeroDivisionError:
                    #handles a combination with a 'perfect' fit, which causes RLM to divide by zero
                    chisq = 0
                    fit = np.polyfit(ys_clean,xs_clean,1)
                    x_fit = list(np.poly1d(fit)(z_pts))
                    
        except ZeroDivisionError:
            #handles a combination with a 'perfect' fit, which causes RLM to divide by zero
            chisq = 0
            fit = np.polyfit(ys,xs,1)
            x_fit = list(np.poly1d(fit)(z_pts))
        
        #ignore combinations with slopes that aren't allowed
        if allowed_slope(x_fit,z_pts) == True:
            angle = 0
            if chisq/dof < chisq_best and common(xs,x_best) == 100 and opposites(xs,x_best) == 100:
                #executes if a good fit is found that is for a unique muon not already in the list
                chisquares.append(chisq/dof)
                x_best.append(xs)
                y_best.append(ys)
                x_fits.append(x_fit)
                cleaned.append(clean)
            elif chisq/dof < chisq_best:
                #executes if a better fit for a muon already in the list is found
                #preference to save original lists of coordinates rather than 'cleaned' ones
                try:
                    #replaces a track with an acceptable un-cleaned version or slightly modified version with a better chi2
                    if clean < cleaned[common(xs,x_best)]:
                        i = common(xs,x_best)
                    elif clean == cleaned[common(xs,x_best)] and chisq/dof < chisquares[common(xs,x_best)]:
                        i = common(xs,x_best)
                    else:
                        continue
                except:
                    #replaces a track with an acceptable un-cleaned version or mirrored version with a better chi2
                    if clean < cleaned[opposites(xs,x_best)]:
                        i = opposites(xs,x_best)
                    elif clean == cleaned[opposites(xs,x_best)] and chisq/dof < chisquares[opposites(xs,x_best)]:
                        i = opposites(xs,x_best)
                    else:
                        continue
                        
                #remove the old entry with the same muon if it had a lower chi2 than the new one
                chisquares = list(chisquares).remove(chisquares[i])
                x_best = list(x_best).remove(x_best[i])
                y_best = list(y_best).remove(y_best[i])
                x_fits = list(x_fits).remove(x_fits[i])
                cleaned = list(cleaned).remove(cleaned[i])
                #and add the new entry in its place
                try:                   
                    chisquares.append(chisq/dof)
                    x_best.append(xs)
                    y_best.append(ys)
                    x_fits.append(x_fit)
                    cleaned.append(clean)
                except AttributeError: 
                    #append doesn't work on empty lists, so this executes if the lists were empty after removal
                    chisquares = [chisq/dof]
                    x_best = [xs]
                    y_best = [ys]
                    x_fits = [x_fit]
                    cleaned = [clean]
                    
    return x_best,y_best,x_fits,chisquares,angle

#helper function used to draw cell outlines on plots
def plotlines(xpts,ypts,ax):
    for x in xpts:
        arr = [x,x]
        ax.plot(arr,ypts,color = 'black')

#creates local reconstructions from the output of process_hits.py
#includes option to plot results and set range of events to reconstruct
def local_reconstruction_xleft_xright(path,start = 0,end = None,plot = False):
    #initialize these to hold reconstructed segments and their chi2 values
    df= pd.DataFrame()
    chis = []
    
    # a few variables to keep track of how many events fail and why
    accepted = 0 #count of accepted events
    rej_count = 0 #count of events which had an unacceptable number of hits
    badchisq = 0 #count of events which had an unacceptable chi2
    badangle = 0 #count of events which had an unacceptable angle
    badch1 = badch2 = badch3 = badch4 = 0
    event = 0 #keeps track of # of events processed
    
    with open(path) as fp:
        if end == None:
            for i, line in enumerate(fp):
            #convert each line of text file (ie. event) to a readable list of the relevant data
            #format of text file must be Event Number,# of Hits,ORBIT_CNT for event,<SL,LAYER,XLEFT,XRIGHT,ZPOS for each hit>
                event += 1
                ch1 = []
                ch2 = []
                ch3 = []
                ch4 = []
                data = line.split(' ')
                orbit = data[2]
                data = data[3:]
                n = len(data)
                index = 0
                j0 = j1 = j2 = j3 = 0
                pts0 = np.zeros((2*n,2))
                pts1 = np.zeros((2*n,2))
                pts2 = np.zeros((2*n,2))
                pts3 = np.zeros((2*n,2))
                while index < n:
                    #make an array of hits for each SL
                    if float(data[index]) == 0:
                        pts0[j0,0] = float(data[index+2])-jitter
                        pts0[j0+1,0] = float(data[index+3])+jitter
                        pts0[j0,1] = pts0[j0+1,1] = float(data[index+4])
                        index +=5
                        j0 += 2
                    elif float(data[index]) == 1.:
                        pts1[j1,0] = float(data[index+2])-jitter
                        pts1[j1+1,0] = float(data[index+3])+jitter
                        pts1[j1,1] = pts1[j1+1,1] = float(data[index+4])
                        index +=5
                        j1 += 2
                    elif float(data[index]) == 2.:
                        pts2[j2,0] = float(data[index+2])-jitter
                        pts2[j2+1,0] = float(data[index+3])+jitter
                        pts2[j2,1] = pts2[j2+1,1] = float(data[index+4])
                        index +=5
                        j2 += 2
                    else:
                        pts3[j3,0] = float(data[index+2])-jitter
                        pts3[j3+1,0] = float(data[index+3])+jitter
                        pts3[j3,1] = pts3[j3+1,1] = float(data[index+4])
                        index +=5
                        j3 += 2
                
                #put the coordinates of the hits in each chamber into a dataframe
                pts0 = pd.DataFrame(removezeros(pts0),columns = ('x','y'))
                pts1 = pd.DataFrame(removezeros(pts1),columns = ('x','y'))
                pts2 = pd.DataFrame(removezeros(pts2),columns = ('x','y'))
                pts3 = pd.DataFrame(removezeros(pts3),columns = ('x','y'))
                
                #do a local reconstruction for each chamber that has a number of hits in the allowed range
                if 3 <= len(pts0)/2 <= max_hits:
                    x0,z0,fit0,chi0,ang0 = find_fit(pts0)
                    chis.extend(chi0)
                    if len(x0) == 0 and ang0 == 1:
                        #executes if no fit was found due to unacceptable angles
                        badangle += 1
                    elif len(x0) == 0 and ang0 == 0:
                        #executes if no fit was found due to unacceptable chi2 values
                        badch1 += 1
                        badchisq += 1
                    else:
                        #executes if acceptable fit(s) were found and adds them to an array
                        accepted += 1
                        for i in np.arange(len(x0)):
                            arr = np.array(fit0[i]+z_pts)
                            arr = np.insert(arr,0,orbit).flatten()
                            ch1.append(arr)
                else: 
                    #executes if the number of hits wasn't in the allowed range
                    rej_count += 1 
                    
                if 3 <= len(pts1)/2 <= max_hits:
                    x1,z1,fit1,chi1,ang1 = find_fit(pts1)          
                    chis.extend(chi1)
                    if len(x1) == 0 and ang1 == 1:
                        badangle += 1
                    elif len(x1) == 0 and ang1 == 0:
                        badch2 += 1
                        badchisq += 1
                    else:
                        accepted += 1
                        for i in np.arange(len(x1)):
                            arr = np.array(fit1[i]+list(np.asarray(z_pts)+4*ZCELL))
                            arr = np.insert(arr,0,orbit).flatten()
                            ch2.append(arr)
                else:
                    rej_count += 1
                    
                if 3 <= len(pts2)/2 <= max_hits:
                    x2,z2,fit2,chi2,ang2 = find_fit(pts2)                 
                    chis.extend(chi2)
                    if len(x2) == 0 and ang2 == 1:
                        badangle += 1
                    elif len(x2) == 0 and ang2 == 0:
                        badch3 += 1
                        badchisq += 1
                    else:
                        accepted += 1
                        for i in np.arange(len(x2)):
                            arr = np.array(fit2[i]+list(np.asarray(z_pts)+Z_SEP))
                            arr = np.insert(arr,0,orbit).flatten()
                            ch3.append(arr)
                else:
                    rej_count += 1  
                    
                if 3 <= len(pts3)/2 <= max_hits:
                    x3,z3,fit3,chi3,ang3 = find_fit(pts3)
                    chis.extend(chi3)
                    if len(x3) == 0 and ang3 == 1:
                        badangle += 1
                    elif len(x3) == 0 and ang3 == 0:
                        badch4 += 1
                        badchisq += 1
                    else:
                        accepted += 1
                        for i in np.arange(len(x3)):
                            arr = np.array(fit3[i]+list(np.asarray(z_pts)+4*ZCELL+Z_SEP))
                            arr = np.insert(arr,0,orbit).flatten()
                            ch4.append(arr)
                else:
                    rej_count += 1
                
                #remove entries where no good fit was found
                ch1 = list(filter(lambda x: len(x) > 1, ch1))
                ch2 = list(filter(lambda x: len(x) > 1, ch2))
                ch3 = list(filter(lambda x: len(x) > 1, ch3))
                ch4 = list(filter(lambda x: len(x) > 1, ch4))
                    
        #if all 4 chambers had an acceptable fit,add all combinations of local reconstructions to a dataframe
                if len(ch1) > 0 and len(ch2) > 0 and len(ch3) > 0 and len(ch4) > 0:
                    chambs = [ch1,ch2,ch3,ch4]
                    combos = np.array(list(itertools.product(*chambs)))
                    for combo in combos:
                        for i in np.arange(len(combo)):
                            df2 = pd.DataFrame(combo[i],dtype = float)
                            df = df.append(df2.T,ignore_index = True)
        else:
            #Does the same thing as above, but for limited range of events. Allows for plotting.
            for i, line in enumerate(fp):
                if start <= i < end:
                #convert each line of text file (ie. event) to a readable list of the relevant data
                    event +=1
                    ch1 = []
                    ch2 = []
                    ch3 = []
                    ch4 = []
                    data = line.split(' ')
                    orbit = data[2]
                    data = data[3:]
                    n = len(data)
                    index = 0
                    j0 = j1 = j2 = j3 = 0
                    pts0 = np.zeros((2*n,2))
                    pts1 = np.zeros((2*n,2))
                    pts2 = np.zeros((2*n,2))
                    pts3 = np.zeros((2*n,2))
                    while index < n:
                        #make an array of hits for each SL
                        if float(data[index]) == 0:
                            pts0[j0,0] = float(data[index+2])-jitter
                            pts0[j0+1,0] = float(data[index+3])+jitter
                            pts0[j0,1] = pts0[j0+1,1] = float(data[index+4])
                            index +=5
                            j0 += 2
                        elif float(data[index]) == 1.:
                            pts1[j1,0] = float(data[index+2])-jitter
                            pts1[j1+1,0] = float(data[index+3])+jitter
                            pts1[j1,1] = pts1[j1+1,1] = float(data[index+4])
                            index +=5
                            j1 += 2
                        elif float(data[index]) == 2.:
                            pts2[j2,0] = float(data[index+2])-jitter
                            pts2[j2+1,0] = float(data[index+3])+jitter
                            pts2[j2,1] = pts2[j2+1,1] = float(data[index+4])
                            index +=5
                            j2 += 2
                        else:
                            pts3[j3,0] = float(data[index+2])-jitter
                            pts3[j3+1,0] = float(data[index+3])+jitter
                            pts3[j3,1] = pts3[j3+1,1] = float(data[index+4])
                            index +=5
                            j3 += 2
                    
                    #put the coordinates of the hits in each chamber into a dataframe
                    pts0 = pd.DataFrame(removezeros(pts0),columns = ('x','y'))
                    pts1 = pd.DataFrame(removezeros(pts1),columns = ('x','y'))
                    pts2 = pd.DataFrame(removezeros(pts2),columns = ('x','y'))
                    pts3 = pd.DataFrame(removezeros(pts3),columns = ('x','y'))
                    
                    if plot:
                        #initialize some stuff for the plots
                        #plots the hits,line of fit, and cell outlines for each chamber
                        fig1,axes = plt.subplots(nrows = 2,ncols = 2,figsize =(10,4),constrained_layout=True)
                        xodd = np.arange(16)*XCELL
                        xeven = np.arange(16)*XCELL+XCELL/2
                        y = [[0,ZCELL],[ZCELL,ZCELL*2],[ZCELL*2,ZCELL*3],[ZCELL*3,ZCELL*4]]
                        
                    if 3 <= len(pts0)/2 <= max_hits:
                        x0,z0,fit0,chi0,ang0 = find_fit(pts0)
                        chis.extend(chi0)
                        if len(x0) == 0 and ang0 == 1:
                            badangle += 1
                        elif len(x0) == 0 and ang0 == 0:
                            badch1 += 1
                            badchisq += 1
                        elif plot:
                            #an added option from above to create the plot for the chamber
                            for pt1 in np.arange(len(xodd)-1):
                                axes[0,0].plot(xodd[pt1+1],z_pts[0],'k.')
                                axes[0,0].plot(xodd[pt1+1],z_pts[2],'k.')

                            for pt2 in np.arange(len(xeven)-1):
                                axes[0,0].plot(xeven[pt2],z_pts[1],'k.')
                                axes[0,0].plot(xeven[pt2],z_pts[3],'k.')
                            
                            accepted += 1
                            for i in np.arange(len(x0)):
                                arr = np.array(fit0[i]+z_pts)
                                arr = np.insert(arr,0,orbit).flatten()
                                ch1.append(arr)
                                
                                for j in np.arange(len(y)):
                                    if j%2 == 0:
                                        plotlines(xeven,y[j],axes[0,0])
                                    else:
                                        plotlines(xodd,y[j],axes[0,0])
                                axes[0,0].hlines([0,13,26,39,52],0,693)
                                
                                axes[0,0].plot(fit0[i],z_pts,label = 'Chi2: '+str(np.round(chi0[i],2)))
                                axes[0,0].scatter(x0[i],z0[i],marker = 'x')
                                axes[0,0].set_title('Orbit '+str(orbit)+' Chamber 1')
                                axes[0,0].set_xlabel('X')
                                axes[0,0].set_ylabel('Z')
                                axes[0,0].set_ylim(0, 53)
                                axes[0,0].set_yticks([0,13,26,39,52])
                                axes[0,0].legend(loc = 'best')
                                if len(x0)== 1:
                                    axes[0,0].set_xlim(min(x0[i])-21,max(x0[i])+21)
                                else:
                                    axes[0,0].set_xlim(min(x0[i])-100,max(x0[i])+100)
                        else:
                            accepted += 1
                            for i in np.arange(len(x0)):
                                arr = np.array(fit0[i]+z_pts)
                                arr = np.insert(arr,0,orbit).flatten()
                                ch1.append(arr)
                    else:
                        rej_count += 1
                        
                    if 3 <= len(pts1)/2 <= max_hits:
                        x1,z1,fit1,chi1,ang1 = find_fit(pts1)
                        chis.extend(chi1)
                        if len(x1) == 0 and ang1 == 1:
                            badangle += 1
                        elif len(x1) == 0 and ang1 == 0:
                            badch2 += 1
                            badchisq += 1
                        elif plot:
                            for pt1 in np.arange(len(xodd)-1):
                                axes[0,1].plot(xodd[pt1+1],z_pts[0],'k.')
                                axes[0,1].plot(xodd[pt1+1],z_pts[2],'k.')

                            for pt2 in np.arange(len(xeven)-1):
                                axes[0,1].plot(xeven[pt2],z_pts[1],'k.')
                                axes[0,1].plot(xeven[pt2],z_pts[3],'k.')
                                
                            accepted += 1
                            for i in np.arange(len(x1)):
                                arr = np.array(fit1[i]+list(np.asarray(z_pts)+4*ZCELL))
                                arr = np.insert(arr,0,orbit).flatten()
                                ch2.append(arr)
                                
                                for j in np.arange(len(y)):
                                    if j%2 == 0:
                                        plotlines(xeven,y[j],axes[0,1])
                                        for pt1 in np.arange(len(xodd)):
                                            axes[0,1].plot(xodd[pt1],z_pts[0],color = 'black')
                                            axes[0,1].plot(xodd[pt1],z_pts[2],color = 'black')
                                    else:
                                        plotlines(xodd,y[j],axes[0,1])
                                        for pt2 in np.arange(len(xeven)):
                                            axes[0,1].plot(xeven[pt2],z_pts[1],color = 'black')
                                            axes[0,1].plot(xeven[pt2],z_pts[3],color = 'black')
                                axes[0,1].hlines([0,13,26,39,52],0,693)
                                
                                axes[0,1].plot(fit1[i],z_pts,label = 'Chi2: '+str(np.round(chi1[i],2)))
                                axes[0,1].scatter(x1[i],z1[i],marker = 'x')
                                axes[0,1].set_title('Orbit '+str(orbit)+' Chamber 2') 
                                axes[0,1].set_xlabel('Y')
                                axes[0,1].set_ylabel('Z')
                                axes[0,1].set_ylim(0, 53)
                                axes[0,1].set_yticks([0,13,26,39,52])
                                axes[0,1].legend(loc = 'best')
                                if len(x1) == 1:
                                    axes[0,1].set_xlim(min(x1[i])-21,max(x1[i])+21)
                                else:
                                    #sets a wider window if multiple tracks were reconstructed so the plot shows everything
                                    axes[0,1].set_xlim(min(x1[i])-100,max(x1[i])+100)
                        else:
                            accepted += 1
                            for i in np.arange(len(x1)):
                                arr = np.array(fit1[i]+list(np.asarray(z_pts)+4*ZCELL))
                                arr = np.insert(arr,0,orbit).flatten()
                                ch2.append(arr)
                    else:
                        rej_count += 1
                        
                    if 3 <= len(pts2)/2 <= max_hits:
                        x2,z2,fit2,chi2,ang2 = find_fit(pts2)
                        chis.extend(chi2)
                        if len(x2) == 0 and ang2 == 1:
                            badangle += 1
                        elif len(x2) == 0 and ang2 == 0:
                            badch3 += 1
                            badchisq += 1
                        elif plot:
                            for pt1 in np.arange(len(xodd)-1):
                                axes[1,0].plot(xodd[pt1+1],z_pts[0],'k.')
                                axes[1,0].plot(xodd[pt1+1],z_pts[2],'k.')

                            for pt2 in np.arange(len(xeven)-1):
                                axes[1,0].plot(xeven[pt2],z_pts[1],'k.')
                                axes[1,0].plot(xeven[pt2],z_pts[3],'k.')
                            accepted += 1
                            for i in np.arange(len(x2)):
                                arr = np.array(fit2[i]+list(np.asarray(z_pts)+Z_SEP))
                                arr = np.insert(arr,0,orbit).flatten()
                                ch3.append(arr)
                                
                                for j in np.arange(len(y)):
                                    if j%2 == 0:
                                        plotlines(xeven,y[j],axes[1,0])
                                        for pt1 in np.arange(len(xodd)):
                                            axes[1,0].plot(xodd[pt1],z_pts[0],color = 'black')
                                            axes[1,0].plot(xodd[pt1],z_pts[2],color = 'black')
                                    else:
                                        plotlines(xodd,y[j],axes[1,0])
                                        for pt2 in np.arange(len(xeven)):
                                            axes[1,0].plot(xeven[pt2],z_pts[1],color = 'black')
                                            axes[1,0].plot(xeven[pt2],z_pts[3],color = 'black')
                                axes[1,0].hlines([0,13,26,39,52],0,693)
                                
                                axes[1,0].plot(fit2[i],z_pts,label = 'Chi2: '+str(np.round(chi2[i],2)))
                                axes[1,0].scatter(x2[i],z2[i],marker = 'x')
                                axes[1,0].set_title('Orbit '+str(orbit)+' Chamber 3')
                                axes[1,0].set_xlabel('X')
                                axes[1,0].set_ylabel('Z')
                                axes[1,0].set_ylim(0, 53)
                                axes[1,0].set_yticks([0,13,26,39,52])
                                axes[1,0].legend(loc = 'best')
                                if len(x2) == 1:
                                    axes[1,0].set_xlim(min(x2[i])-21,max(x2[i])+21)
                                else:
                                    axes[1,0].set_xlim(min(x2[i])-100,max(x2[i])+100)
                        else:
                            accepted += 1
                            for i in np.arange(len(x2)):
                                arr = np.array(fit2[i]+list(np.asarray(z_pts)+Z_SEP))
                                arr = np.insert(arr,0,orbit).flatten()
                                ch3.append(arr)
                    else:
                        rej_count += 1
                        
                    if 3 <= len(pts3)/2 <= max_hits:
                        x3,z3,fit3,chi3,ang3 = find_fit(pts3)
                        chis.extend(chi3)
                        if len(x3) == 0 and ang3 == 1:
                            badangle += 1
                        elif len(x3) == 0 and ang3 == 0:
                            badch4 += 1
                            badchisq += 1
                        elif plot:
                            for pt1 in np.arange(len(xodd)-1):
                                axes[1,1].plot(xodd[pt1+1],z_pts[0],'k.')
                                axes[1,1].plot(xodd[pt1+1],z_pts[2],'k.')

                            for pt2 in np.arange(len(xeven)-1):
                                axes[1,1].plot(xeven[pt2],z_pts[1],'k.')
                                axes[1,1].plot(xeven[pt2],z_pts[3],'k.')
                            accepted += 1
                            for i in np.arange(len(x3)):
                                arr = np.array(fit3[i]+list(np.asarray(z_pts)+4*ZCELL+Z_SEP))
                                arr = np.insert(arr,0,orbit).flatten()
                                ch4.append(arr)
                                
                                for j in np.arange(len(y)):
                                    if j%2 == 0:
                                        plotlines(xeven,y[j],axes[1,1])
                                        for pt1 in np.arange(len(xodd)):
                                            axes[1,1].plot(xodd[pt1],z_pts[0],color = 'black')
                                            axes[1,1].plot(xodd[pt1],z_pts[2],color = 'black')
                                    else:
                                        plotlines(xodd,y[j],axes[1,1])
                                        for pt2 in np.arange(len(xeven)):
                                            axes[1,1].plot(xeven[pt2],z_pts[1],color = 'black')
                                            axes[1,1].plot(xeven[pt2],z_pts[3],color = 'black')
                                axes[1,1].hlines([0,13,26,39,52],0,693)
                                
                                axes[1,1].plot(fit3[i],z_pts,label = 'Chi2: '+str(np.round(chi3[i],2)))
                                axes[1,1].scatter(x3[i],z3[i],marker = 'x')
                                axes[1,1].set_title('Orbit '+str(orbit)+' Chamber 4')
                                axes[1,1].set_xlabel('Y')
                                axes[1,1].set_ylabel('Z')
                                axes[1,1].set_ylim(0, 53)
                                axes[1,1].set_yticks([0,13,26,39,52])
                                axes[1,1].legend(loc = 'best')
                                if len(x3) == 1:
                                    axes[1,1].set_xlim(min(x3[i])-21,max(x3[i])+21)
                                else:
                                    axes[1,1].set_xlim(min(x3[i])-100,max(x3[i])+100)
                        else:
                            accepted += 1
                            for i in np.arange(len(x3)):
                                arr = np.array(fit3[i]+list(np.asarray(z_pts)+4*ZCELL+Z_SEP))
                                arr = np.insert(arr,0,orbit).flatten()
                                ch4.append(arr)
                    else:
                        rej_count += 1
                        
                    #remove entries where no good fit was found
                    ch1 = list(filter(lambda x: len(x) > 1, ch1))
                    ch2 = list(filter(lambda x: len(x) > 1, ch2))
                    ch3 = list(filter(lambda x: len(x) > 1, ch3))
                    ch4 = list(filter(lambda x: len(x) > 1, ch4))
                    
        #if all 4 chambers had an acceptable fit,add all combinations of local reconstructions to a dataframe
                    if len(ch1) > 0 and len(ch2) > 0 and len(ch3) > 0 and len(ch4) > 0:
                        chambs = [ch1,ch2,ch3,ch4]
                        combos = np.array(list(itertools.product(*chambs)))
                        for combo in combos:
                            for i in np.arange(len(combo)):
                                df2 = pd.DataFrame(combo[i],dtype = float)
                                df = df.append(df2.T,ignore_index = True)
                  
                    plt.show()
                    plt.close('all')
                            
    fp.close()
    print('Events With Acceptable Local Reconstructions: '+str(accepted)+' out of '+str(event*4))
    print('Events that had a number of hits outside of the range: '+str(rej_count))
    print('Reconstructions With Angles above the Maximum: '+str(badangle))
    print('Chambers with reconstructions above the Chi Squared Threshold: '+str(badchisq))
    #un-comment the lines below if you want to know how many chi2 failures wre in each individual chamber
    #print('Failed Reconstructions in Chamber 1: '+str(badch1))
    #print('Failed Reconstructions in Chamber 2: '+str(badch2))
    #print('Failed Reconstructions in Chamber 3: '+str(badch3))
    #print('Failed Reconstructions in Chamber 4: '+str(badch4))
    return df,chis,badch1,badch2,badch3,badch4

#creates global reconstructions from the local reconstruction dataframe
#includes different options for plotting and range of reconstruction
def total_reconstruction(df,start = 0,end = None,split = False,plot2d = False,plot3d = False):
    count = len(df.index)
    accepted = 0
    if split: #plots each reconstruction separately
        if end == None:
            end = count
        j = 0
        chis3d = []
        chis = []
        while j < count:
            if start*4 <= j < end*4:
                #iterate through the dataframe, creating a line of best fit across all chambers
                ch1 = list(df.loc[j,:].dropna())
                ch2 = list(df.loc[j+1,:].dropna())
                ch3 = list(df.loc[j+2,:].dropna())
                ch4 = list(df.loc[j+3,:].dropna())
                x1 = ch1[1:len(ch1)//2+1]
                z1 = ch1[len(ch1)//2+1:]
                y2 = ch2[1:len(ch2)//2+1]
                z2 = ch2[len(ch2)//2+1:]
                #create fits to assign y-coordinates to hits in chamber 1 and x-coordinates to hits in chamber 2
                xfit1 = np.polyfit(z1,x1,1)
                yfit1 = np.polyfit(z2,y2,1)
                y1 = list(np.poly1d(yfit1)(z1))
                x2 = list(np.poly1d(xfit1)(z2))

                x3 = ch3[1:len(ch3)//2+1]
                z3 = ch3[len(ch3)//2+1:]
                y4 = ch4[1:len(ch4)//2+1]
                z4 = ch4[len(ch4)//2+1:]
                #create fits to assign y-coordinates to hits in chamber 3 and x-coordinates to hits in chamber 4
                xfit2 = np.polyfit(z3,x3,1)
                yfit2 = np.polyfit(z4,y4,1)
                y3 = list(np.poly1d(yfit2)(z3))
                x4 = list(np.poly1d(xfit2)(z4))
                event_nr = int(ch1[0]) #saves the orbit number to use as an event label

                if plot2d:
                    #plots the fit between the points of chambers 1 and 3 (x-z plane) and chambers 2 and 4 (y-z plane)
                    #initialize figures and plot hits
                    fig1,ax1 = plt.subplots(figsize =(6,6))
                    fig2,ax2 = plt.subplots(figsize =(6,6))
                    ax1.scatter(x1,z1, color = 'b',label = 'Chamber 1')
                    ax1.scatter(x3,z3, color = 'g',label = 'Chamber 3')
                    ax1.set_title('Orbit '+str(event_nr)+' x-z fit')
                    ax2.scatter(y2,z2,color = 'b',label = 'Chamber 2')
                    ax2.scatter(y4,z4,color = 'g',label = 'Chamber 4')
                    ax2.set_title('Orbit '+str(event_nr)+' y-z fit')
                    
                x1new = x1+x3
                z1new = z1+z3
                y1new,chisq1,_,_,_ = np.polyfit(x1new,z1new,1,full = True)
                
                y2new = y2+y4
                z2new = z2+z4
                x2new,chisq2,_,_,_ = np.polyfit(y2new,z2new,1,full = True)
                chis.extend([float(chisq1/2),float(chisq2/2)])
                
                if plot2d:
                    #plot lines of best fit and label axes
                    ax1.plot(x1new,np.poly1d(y1new)(x1new),color = 'r',label = 'fit: '+str(chisq1/2))
                    ax1.legend(loc = 'best')
                    ax2.plot(y2new,np.poly1d(x2new)(y2new),color = 'r',label = 'fit: '+str(chisq2/2))
                    ax2.legend(loc = 'best')
                    ax1.set_xlabel('X')
                    ax1.set_ylabel('Z')
                    ax2.set_xlabel('Y')
                    ax2.set_ylabel('Z')
                
                #only accept events with good agreement between corresponding chambers
                if float(chisq1/2) < cut2d and float(chisq2/2) < cut2d:
                    x1.extend(x2)
                    x1.extend(x3)
                    x1.extend(x4)
                    y1.extend(y2)
                    y1.extend(y3)
                    y1.extend(y4)
                    z1.extend(z2)
                    z1.extend(z3)
                    z1.extend(z4)

                    #find a plane of best fit for the x-z axis and y-z axis
                    A_xz = np.vstack((x1, np.ones(len(x1)))).T
                    m_xz,chix,_,_ = np.linalg.lstsq(A_xz, z1,rcond=None)
                    A_yz = np.vstack((y1, np.ones(len(y1)))).T
                    m_yz,chiy,_,_ = np.linalg.lstsq(A_yz, z1,rcond=None)
                    chis3d.extend([float(chix/3),float(chiy/3)])

                    #only accept events with good planes of best fit
                    if chix/3 < cut3d and chiy/3 < cut3d:
                        accepted += 1
                        #calculate points along the intersection of the planes to get best global fit
                        z_final = np.linspace(0,888)
                        x_final = (z_final - m_xz[1])/m_xz[0]
                        y_final = (z_final - m_yz[1])/m_yz[0]
                        label = 'Orbit '+str(event_nr)
                        fig = plt.figure(figsize =(6,6))
                        ax = Axes3D(fig)
                        ax.set_xlabel("X")
                        ax.set_ylabel("Y")
                        ax.set_zlabel("Z")
                        ax.scatter(x1,y1,z1)
                        ax.plot(x_final,y_final,z_final,label = label)
                        ax.set_title(label)
                        ax.set_xlim(0, 693)
                        ax.set_ylim(0, 693)
                        ax.set_proj_type('ortho')
                        plt.show()
            j += 4
    else:
    #either skip plotting or plot all reconstructions together
        if plot3d:
            #initialize figure for plotting all reconstructions together
            fig = plt.figure(figsize =(8,8))
            ax = Axes3D(fig)
            ax.set_xlabel("X")
            ax.set_ylabel("Y")
            ax.set_zlabel("Z")
        if end == None:
            end = count
        j = 0
        chis3d = []
        chis = []
        while j < count:
            if start*4 <= j < end*4:
                #iterate through the dataframe, creating a line of best fit across all chambers
                ch1 = list(df.loc[j,:].dropna())
                ch2 = list(df.loc[j+1,:].dropna())
                ch3 = list(df.loc[j+2,:].dropna())
                ch4 = list(df.loc[j+3,:].dropna())
                x1 = ch1[1:len(ch1)//2+1]
                z1 = ch1[len(ch1)//2+1:]
                y2 = ch2[1:len(ch2)//2+1]
                z2 = ch2[len(ch2)//2+1:]
                #create fits to assign y-coordinates to hits in chamber 1 and x-coordinates to hits in chamber 2
                xfit1 = np.polyfit(z1,x1,1)
                yfit1 = np.polyfit(z2,y2,1)
                y1 = list(np.poly1d(yfit1)(z1))
                x2 = list(np.poly1d(xfit1)(z2))

                x3 = ch3[1:len(ch3)//2+1]
                z3 = ch3[len(ch3)//2+1:]
                y4 = ch4[1:len(ch4)//2+1]
                z4 = ch4[len(ch4)//2+1:]
                #create fits to assign y-coordinates to hits in chamber 3 and x-coordinates to hits in chamber 4
                xfit2 = np.polyfit(z3,x3,1)
                yfit2 = np.polyfit(z4,y4,1)
                y3 = list(np.poly1d(yfit2)(z3))
                x4 = list(np.poly1d(xfit2)(z4))
                event_nr = int(ch1[0])
                
                if plot2d:
                    #plots the fit between the points of chambers 1 and 3 (x-z plane) and chambers 2 and 4 (y-z plane)
                    #initialize figures and plot hits
                    fig1,ax1 = plt.subplots(figsize =(6,6))
                    fig2,ax2 = plt.subplots(figsize =(6,6))
                    ax1.scatter(x1,z1, color = 'b',label = 'Chamber 1')
                    ax1.scatter(x3,z3, color = 'g',label = 'Chamber 3')
                    ax1.set_title('Orbit '+str(event_nr)+' x-z fit')
                    ax2.scatter(y2,z2,color = 'b',label = 'Chamber 2')
                    ax2.scatter(y4,z4,color = 'g',label = 'Chamber 4')
                    ax2.set_title('Orbit '+str(event_nr)+' y-z fit')
                
                x1new = x1+x3
                z1new = z1+z3
                y1new,chisq1,_,_,_ = np.polyfit(x1new,z1new,1,full = True)
                
                y2new = y2+y4
                z2new = z2+z4
                x2new,chisq2,_,_,_ = np.polyfit(y2new,z2new,1,full = True)
                chis.extend([float(chisq1/2),float(chisq2/2)])
                
                if plot2d:
                    #plot lines of best fit and label axes
                    ax1.plot(x1new,np.poly1d(y1new)(x1new),color = 'r',label = 'fit: '+str(chisq1/2))
                    ax1.legend(loc = 'best')
                    ax2.plot(y2new,np.poly1d(x2new)(y2new),color = 'r',label = 'fit: '+str(chisq2/2))
                    ax2.legend(loc = 'best')
                    ax1.set_xlabel('X')
                    ax1.set_ylabel('Z')
                    ax2.set_xlabel('Y')
                    ax2.set_ylabel('Z')
                
                #only accept events with good agreement between corresponding chambers
                if float(chisq1/2) < cut2d and float(chisq2/2) < cut2d:
                    x1.extend(x2)
                    x1.extend(x3)
                    x1.extend(x4)
                    y1.extend(y2)
                    y1.extend(y3)
                    y1.extend(y4)
                    z1.extend(z2)
                    z1.extend(z3)
                    z1.extend(z4)

                    #find a plane of best fit for the x-z axis and y-z axis
                    A_xz = np.vstack((x1, np.ones(len(x1)))).T
                    m_xz,chix,_,_ = np.linalg.lstsq(A_xz, z1,rcond=None)
                    A_yz = np.vstack((y1, np.ones(len(y1)))).T
                    m_yz,chiy,_,_ = np.linalg.lstsq(A_yz, z1,rcond=None)
                    chis3d.extend([float(chix/3),float(chiy/3)])

                    #only accept events with good planes of best fit
                    if chix/3 < cut3d and chiy/3 < cut3d:
                        accepted += 1
                        #calculate points along the intersection of the planes to get best global fit
                        z_final = np.linspace(0,888)
                        x_final = (z_final - m_xz[1])/m_xz[0]
                        y_final = (z_final - m_yz[1])/m_yz[0]
                        if plot3d:
                            label = 'Orbit '+str(event_nr)
                            ax.scatter(x1,y1,z1)
                            ax.plot(x_final,y_final,z_final,label = label)
                            ax.legend(loc = 'upper left')
                            ax.set_xlim(0, 693)
                            ax.set_ylim(0, 693)
                            plt.show()

            j+=4
    print('Events With Acceptable Global Reconstruction: '+str(accepted)+' out of '+str(count//4))
    return chis,chis3d

In [ ]:
#Keep in mind that if you are plotting something without setting a start/end,you will be generating a LOT of plots!

#Set a start and end to only locally reconstruct a certain range of events
#Set plot = True to display local reconstruction plots
#chi2 for local reconstructions and # of failures per chamber given as output in case you want to look
data,chi_local,badch1,badch2,badch3,badch4 = local_reconstruction_xleft_xright('<insert file path here>')

#if you are using jupyter notebook and want to use plot3d, you have to un-comment the line below
#Note: Local reconstructions will not display if you ran the code with it un-commented,restart the kernel and comment it out again to fix this
#%matplotlib notebook

#Set a start and end to only globally reconstruct a certain range of locally reconstructed events
#Set split = True for global reconstructions,each plotted on its own figure
#Set plot2d = True to see plots of the x-z and y-z plane fits for each event
#Set plot3d = True for global reconstructions,all plotted together on the same figure
#as mentioned above,local reconstructions and plot3d are not compatable; if you want both local and global reconstructions simultaneously,use split instead
#residuals values for 2d and 3d global reconstrucions given as output in case you want to look
chi_2d,chi_global = total_reconstruction(data)